## Layer-wise Relevance Propagation  for sentiment analysis

In [1]:
from keras.layers import Input
from keras.layers import RNN,Lambda,Dense,Bidirectional,Add,LSTM,Layer,Concatenate,Masking,Dropout,InputLayer
from keras import Model

import keras
from keras.utils.generic_utils import CustomObjectScope

import os,sys
from os.path import dirname, join, abspath
import numpy as np
import pickle

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
UTILS_DIR = '../common_utils/'
sys.path.insert(0, abspath(join(dirname(UTILS_DIR), '.')))
from LRP_utils import LRP_LSTMCell

# Loading pretrained sentiment model

In [5]:
WORKING_DIR = os.getcwd()
PATH = WORKING_DIR +'/models/bi_lstm_model.pickle'

hid_dim = embd_dim = dim = 60
n_classes = 5
with open(PATH,'rb') as m:
    pretrained_weights  = pickle.load(m)

f_W = pretrained_weights['left_encoder_weights']['Wxh_Left'].transpose()
f_U = pretrained_weights['left_encoder_weights']['Whh_Left'].transpose()
f_b = pretrained_weights['left_encoder_weights']['bxh_Left'] + pretrained_weights['left_encoder_weights']['bhh_Left']

f_W[:,dim:2*dim],f_W[:,2*dim:3*dim]= f_W[:,2*dim:3*dim].copy(),f_W[:,dim:2*dim].copy()
f_U[:,dim:2*dim],f_U[:,2*dim:3*dim]= f_U[:,2*dim:3*dim].copy(),f_U[:,dim:2*dim].copy()
f_b[dim:2*dim],f_b[2*dim:3*dim]= f_b[2*dim:3*dim].copy(),f_b[dim:2*dim].copy()

b_W = pretrained_weights['right_encoder_weights']['Wxh_Right'].transpose()
b_U = pretrained_weights['right_encoder_weights']['Whh_Right'].transpose()
b_b = pretrained_weights['right_encoder_weights']['bxh_Right'] + pretrained_weights['right_encoder_weights']['bhh_Right']


b_W[:,dim:2*dim],b_W[:,2*dim:3*dim]= b_W[:,2*dim:3*dim].copy(),b_W[:,dim:2*dim].copy()
b_U[:,dim:2*dim],b_U[:,2*dim:3*dim]= b_U[:,2*dim:3*dim].copy(),b_U[:,dim:2*dim].copy()
b_b[dim:2*dim],b_b[2*dim:3*dim]= b_b[2*dim:3*dim].copy(),b_b[dim:2*dim].copy()

f_out = pretrained_weights['output_weights']['Why_Left'].transpose()
b_out = pretrained_weights['output_weights']['Why_Right'].transpose()

out = np.concatenate((f_out,b_out))
b = np.zeros(n_classes)

# Define Keras Model

In [6]:
with CustomObjectScope({'LRP_LSTMCell': LRP_LSTMCell}):
    cell = LRP_LSTMCell(60,recurrent_activation='sigmoid',implementation=1)
    x = keras.Input((None,60))
    bi_lstm = Bidirectional(RNN(cell))
    h = bi_lstm(x)
    y= Dense(5)(h)
    model = Model(inputs=[x], outputs=[y])
model.set_weights([f_W, f_U, f_b, b_W, b_U, b_b, out, b])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 60)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120)               58080     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 605       
Total params: 58,685
Trainable params: 58,685
Non-trainable params: 0
_________________________________________________________________


# Loading word embeddings

In [7]:
embd_path = WORKING_DIR + '/embeddings/embeddings.npy'
E = np.load(embd_path, mmap_mode='r')

with open(WORKING_DIR+'/embeddings/vocab.dms','rb') as f_voc:
    voc  = pickle.load(f_voc)
    f_voc.close()
    
text_0        = ['i','hate','the','movie','though','the','plot','is','interesting','.']
text_indeces_0 = [voc.index(w) for w in text_0]
text_1 = 'i love the movie though it is so boring .'.split(' ')
text_indeces_1 = [voc.index(w) for w in text_1]

data1 = E[text_indeces_0].reshape((1,len(text_0),dim))
data2 = E[text_indeces_1].reshape((1,len(text_1),dim))
data = np.vstack((data1,data2))
print(model.predict([data]))

[[ 0.83815783  0.55615956 -0.2387495   0.06664614 -0.4852757 ]
 [ 1.7834715   1.8099647   0.25141034 -1.1962156  -2.483652  ]]


# Perform LRP

In [11]:
from LRP_utils import get_target_activation
from LRP_utils import cashe_lstm_forward_activations_batch
from LRP_utils import lrp_dense_batch,lrp_lstm_batch
eps=0.001
bias_factor=0

In [12]:
all_activations = {}
bi_lstm_layer = model.layers[1]#bi-LSTM
dense_layer = model.layers[-1]
all_activations[bi_lstm_layer.name] = cashe_lstm_forward_activations_batch(bi_lstm_layer, data,bidirectional=True,dim=dim)
all_activations[dense_layer.name] = get_target_activation(dense_layer.input,dense_layer.output,all_activations[bi_lstm_layer.name]['final'])

hin = all_activations[bi_lstm_layer.name]['final']
w , b = dense_layer.get_weights()
hout = all_activations[dense_layer.name][0]#.flatten()
bias_nb_units = w.shape[0]

Rout_mask = np.zeros_like(hout)
Rout_mask[0,0] = 1
Rout_mask[1,1] = 1
#Rout_mask[1,LRP_CLASS] = 1
viz_dense = lrp_dense_batch(hin, w, b, hout, hout*Rout_mask, bias_nb_units, eps, bias_factor, dense_layer.name, debug=True)


hin = data
Rout_Left_last = viz_dense[:,:dim].copy()
Rout_Right_last = viz_dense[:,dim:].copy()

seq_len = hin.shape[1]
hidden_dim = dim
input_dim = dim
batch = data.shape[0]
lstm_w = bi_lstm.get_weights()
layer_name = bi_lstm_layer.name
Rx = lrp_lstm_batch(all_activations, hin, Rout_Left_last, batch, seq_len, hidden_dim, input_dim, lstm_w, True, layer_name, Rout_Right_last, eps, bias_factor, debug=True)

dense_1 local diff:  -2.4947244092743404e-07
bidirectional_1 local diff:  4.489909415400581


In [13]:
R = np.sum(Rx,axis=-1)

# Create Saliency heatmap

In [14]:
from IPython.display import display, HTML
from heatmap import html_heatmap

display(HTML(html_heatmap(text_0, R[0])))
display(HTML(html_heatmap(text_1, R[1])))